In [1]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

In [ ]:
import json
import logging
import random

from tqdm.auto import tqdm
from vllm import LLM, SamplingParams

# ロギングの設定
logging.basicConfig(level=logging.INFO)

# 有効なコード翻訳ペア（必要に応じて追加・修正してください）
VALID_TRANSLATION_PAIRS = [
    ("Java", "C"),
    ("Python", "JavaScript"),
    ("C", "Java"),
    ("JavaScript", "Python"),
    ("C++", "Python"),
    # 他のペアも追加可能
]

# 設定定数
CONFIG = {
    "MODEL_NAME": "codellama/CodeLlama-34b-Instruct-hf",
    "TENSOR_PARALLEL_SIZE": 2,  # 利用環境のGPU数に合わせる
    "MAX_NUM_SEQS": 1000,
    "MAX_NUM_BATCHED_TOKENS": 4096,
    "MAX_MODEL_LEN": 4096,
    "DOWNLOAD_DIR": "./cache",
    "SAMPLING_PARAMS": SamplingParams(
        temperature=0.2,
        top_p=0.9,
        max_tokens=1024,
        repetition_penalty=1.1,
        stop=[ "###", "assistant", "user", "<EOD>"],
    ),
    "NUM_BATCHES": 1000,
    "BATCH_SIZE": 1000,
    "OUTPUT_FILE_NAME": "./tanuki_magpie-code-translate.jsonl",
    "BACKUP_FILE_NAME": "./backup.jsonl",
    "BACKUP_FREQUENCY": 2,
    "MIN_TEXT_LENGTH": 10,
    "ENDING_PUNCTUATIONS": ["。", ".", "?", "？"],
}


def initialize_model():
    """vLLMでモデルを初期化する"""
    return LLM(
        model=CONFIG["MODEL_NAME"],
        tensor_parallel_size=CONFIG["TENSOR_PARALLEL_SIZE"],
        max_num_seqs=CONFIG["MAX_NUM_SEQS"],
        max_num_batched_tokens=CONFIG["MAX_NUM_BATCHED_TOKENS"],
        download_dir=CONFIG["DOWNLOAD_DIR"],
        max_model_len=CONFIG["MAX_MODEL_LEN"],
    )


def get_instruction_prompt(source_lang, target_lang):
    """
    ソース言語で実用的なコードを生成するためのプロンプトを作成する。
    ここでは、{source_lang}のコードを生成し、後段で{target_lang}への翻訳に利用します。
    """
    prompt = f"""<s>あなたは熟練したプログラマです。以下のタスクに従い、{source_lang}で実用的なプログラムコードを作成してください。

【タスク】
-（{source_lang}）で動作する、実行可能なコードを作成してください。
- コード以外の解説や文章は出力しないでください。
- このコードは後で{target_lang}への翻訳に使用されます。
- コードは必ず以下の形式で出力してください：

【条件】
- コードは実行可能であること。
- コードの末尾は適切な終端記号（例: 「.」や「;」）で終わること。
- 必ず以下の形式で出力し、最後に ``` で閉じてください。
- 自然言語は生成してはならず、プログラミング言語のみ生成すること。

    ```{source_lang}
    # ここにソースコード
    ```

【出力】
```{source_lang}
"""
    return prompt


def get_output_prompt(source_lang, target_lang, source_code):
    prompt = f"""あなたは熟練したプログラマです。
以下は{source_lang}で書かれたソースコードです。これを{target_lang}のコードに翻訳し、以下の形式で出力してください。

- コードブロックのみを返してください。
- 余計な解説は入れないでください。

【ソースコード】
{source_code}

次の行から翻訳したコードのみを記述してください:
```{target_lang}
"""
    return prompt



def process_batch(batch_size, model):
    """
    1バッチ分の処理を行います。
    ・各サンプルについて、ランダムに翻訳ペアを選択し、ソース言語でのコード生成指示文を作成
    ・生成されたソースコードに対して、指定されたターゲット言語への翻訳を生成
    ・最終的に、OpenAI messages形式（system, user, assistant）でデータを生成
    """
    instruction_prompts = []
    pair_list = []
    for _ in range(batch_size):
        pair = random.choice(VALID_TRANSLATION_PAIRS)
        pair_list.append(pair)
        instruction_prompts.append(get_instruction_prompt(pair[0], pair[1]))

    instruction_outputs = model.generate(instruction_prompts, CONFIG["SAMPLING_PARAMS"])

    valid_instructions = []
    valid_language_pairs = []
    for idx, output in enumerate(instruction_outputs):
        src_lang, tgt_lang = pair_list[idx]
        if not output.outputs:
            continue
        ins_text = output.outputs[0].text.strip()
        if (
            output.outputs[0].finish_reason == "stop"
            and len(ins_text) >= CONFIG["MIN_TEXT_LENGTH"]
            and ins_text[-1] in CONFIG["ENDING_PUNCTUATIONS"]
        ):
            valid_instructions.append(ins_text)
            valid_language_pairs.append((src_lang, tgt_lang))

    logging.info(f"{batch_size}個中、{len(valid_instructions)}個の有効なソースコードを生成しました")

    if not valid_instructions:
        return []

    # 各有効なソースコードに対して、翻訳プロンプトを作成し一括生成
    output_prompts = []
    for (src_lang, tgt_lang), code in zip(valid_language_pairs, valid_instructions):
        output_prompts.append(get_output_prompt(src_lang, tgt_lang, code))

    output_results = model.generate(output_prompts, CONFIG["SAMPLING_PARAMS"])

    results = []
    valid_pairs = 0
    for (src_lang, tgt_lang), src_code, out in zip(valid_language_pairs, valid_instructions, output_results):
        if not out.outputs:
            continue
        out_text = out.outputs[0].text.strip()
        if (
            out.outputs[0].finish_reason == "stop"
            and len(out_text) >= CONFIG["MIN_TEXT_LENGTH"]
            and out_text[-1] in CONFIG["ENDING_PUNCTUATIONS"]
        ):
            messages = [
                {"role": "system", "content": f"{src_lang}から{tgt_lang}へのコード翻訳タスクです。"},
                {"role": "user", "content": src_code},
                {"role": "assistant", "content": out_text},
            ]
            new_data = {
                "messages": messages,
                "source_code": src_code,
                "translated_code": out_text,
                "source_language": src_lang,
                "target_language": tgt_lang,
            }
            results.append(new_data)
            valid_pairs += 1

    logging.info(f"{len(valid_instructions)}個のソースコードに対し、{valid_pairs}個の有効な翻訳を生成しました")
    return results


def save_backup(dataset, file_name):
    """バックアップを保存する関数"""
    with open(file_name, "w", encoding="utf-8") as f:
        for item in dataset:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")


def create_dataset(model, num_batches, batch_size, backup_frequency, backup_file):
    """バッチ毎にデータを処理し、バックアップを保存する"""
    new_dataset = []

    for batch_num in tqdm(range(num_batches)):
        new_data_list = process_batch(batch_size, model)
        new_dataset.extend(new_data_list)

        if (batch_num + 1) % backup_frequency == 0:
            logging.info(f"{batch_num + 1}バッチ目でバックアップを保存します")
            save_backup(new_dataset, backup_file)
        logging.info(f"現在の総データ数: {len(new_dataset)}")

    return new_dataset


def main():
    """メイン処理"""
    model = initialize_model()

    new_dataset = create_dataset(
        model,
        CONFIG["NUM_BATCHES"],
        CONFIG["BATCH_SIZE"],
        CONFIG["BACKUP_FREQUENCY"],
        CONFIG["BACKUP_FILE_NAME"],
    )

    logging.info(f"作成されたデータ数: {len(new_dataset)}")

    # 結果の保存
    with open(CONFIG["OUTPUT_FILE_NAME"], "w", encoding="utf-8") as f:
        for item in new_dataset:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")


if __name__ == "__main__":
    main()


RuntimeError: Failed to import transformers.processing_utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject